In [5]:
from os import environ
import json

from dotenv import find_dotenv, load_dotenv
from elasticsearch import Elasticsearch
import spacy
from spellchecker import SpellChecker
from tqdm import tqdm


load_dotenv(find_dotenv())

True

In [6]:
es = Elasticsearch(
    environ.get('ELASTIC_URL'),
    basic_auth=(environ.get('ELASTIC_USER'), environ.get('ELASTIC_PASSWORD')),
    verify_certs=True
)

if es.ping():
    print("Подключение установлено.")
else:
    print("Не удалось подключиться к Elasticsearch.")

Подключение установлено.


In [7]:
index_name = "documents"

mappings = {
    "mappings": {
        "properties": {
            "document_name": {
                "type": "text"
            },
            "page_number": {
                "type": "integer"
            },
            "paragraph_number": {
                "type": "integer"
            },
            "text": {
                "type": "text"
            },
            "lemmatized_text": {
                "type": "text"
            },
            "bbox": {
                "type": "float"
            }
        }
    }
}

if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mappings)
    print(f"Индекс '{index_name}' создан")
else:
    print(f"Индекс '{index_name}' уже существует")

Индекс 'documents' создан


In [8]:
nlp = spacy.load("ru_core_news_sm")
spell = SpellChecker(language='ru')

In [9]:
def correct_text(text):
    words = text.split()
    corrected_words = []

    for word in words:
        corrected_word = spell.correction(word)
        if corrected_word is None:
            corrected_word = word
        corrected_words.append(corrected_word)
    
    return " ".join(corrected_words)

def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

def preprocess_text(text): return lemmatize_text(correct_text(text))

In [10]:
pdf_path = '../jsons/elasticsearch_data.json'

In [11]:
with open(pdf_path, 'r', encoding='utf-8') as f:
    documents = json.load(f)

In [12]:
page_number = 0
for doc in tqdm(documents[:50]): # всего 150 записей, потому что все записи даже одного документа обрабатывались бы час
    doc["lemmatized_text"] = preprocess_text(doc['text'])
    es.index(index=index_name, document=doc)

100%|██████████| 50/50 [00:58<00:00,  1.17s/it]


In [13]:
query_text = "Железнодорожный транспотр" # ошибка допущена специально

In [17]:
query = {
    "query": {
        "match": {
            "lemmatized_text": {
                "query": preprocess_text(query_text),
                "minimum_should_match": "80%"
            } 
        }
    }
}

response = es.search(index=index_name, body=query)

for hit in response["hits"]["hits"]:
    print(f"Document Name: {hit['_source']['document_name']}")
    print(f"Page Number: {hit['_source']['page_number']}")
    print(f"Paragraph Number: {hit['_source']['paragraph_number']}")
    print(f"bbox: {hit['_source']['bbox']}")
    print(f"Text: {hit['_source']['text']}")
    print("------")


Document Name: 7enYF2uL5kFZlOOpQhLl0nUT91RjCbeR.pdf
Page Number: 1
Paragraph Number: 10
bbox: [70.94400024414062, 570.1958618164062, 527.9199829101562, 678.9252319335938]
Text: обеспечить реализацию Стратегии. 5. Мониторинг реализации Стратегии и ведение транспортно- экономического баланса Российской Федерации возложить на федеральное государственное бюджетное учреждение "Научный центр по комплексным транспортным проблемам Министерства транспорта Российской  Федерации". 
------
Document Name: 7enYF2uL5kFZlOOpQhLl0nUT91RjCbeR.pdf
Page Number: 5
Paragraph Number: 2
bbox: [70.94400024414062, 73.0687255859375, 528.1300048828125, 161.5251922607422]
Text: Стратегические направления развития евразийской экономической  интеграции до 2025 года, утвержденные решением Высшего Евразийского экономического совета от 11 декабря 2020 г. № 12, а также иные акты и решения органов Евразийского экономического союза в части транспорта. 
------


Сохраняю нижний блок для быстрого пересоздания индекса

In [15]:
# response = es.indices.delete(index=index_name, ignore=[400, 404])
# print(response)